
# CNEFE 2022 (categorias 1..8 + subtipos para OUTRAS_FINALIDADES)

Este notebook reproduz o fluxo utilizado para o CNEFE 2022:

- Mapeia `COD_ESPECIE` (1..8) diretamente para colunas 0/1 (`cat_*`).
- Gera subtipos (`sub_*`) **apenas** quando `COD_ESPECIE == 6` (OUTRAS_FINALIDADES), via palavras‑chave em `DSC_ESTABELECIMENTO`.
- Exporta:
  - `outputs/cnefe2022_wide_v3.gpkg` (camada `cnefe2022_wide_v3`)
  - `outputs/cnefe2022_wide_v3.geojson`
  - `outputs/cnefe2022_aggr_face_v3.csv` (agregado por `NUM_FACE`)
  - `outputs/cnefe2022_aggr_setor_v3.csv` (agregado por `COD_SETOR`)

> **Ajustes usuais**: alterar `INPUT_JSON` e editar o dicionário `SUBTIPO_KEYWORDS`.


In [ ]:

# (Opcional) Dependências se estiver em um ambiente "limpo"
# !pip -q install geopandas pyogrio shapely fiona pyproj


In [ ]:

from pathlib import Path
import pandas as pd
import geopandas as gpd

# Caminhos de entrada/saída
INPUT_JSON = Path('/mnt/data/qg_810_endereco_Munic4310009.json')  # ajuste conforme necessário
OUT_DIR    = Path('/mnt/data/outputs'); OUT_DIR.mkdir(parents=True, exist_ok=True)

OUT_GPKG    = OUT_DIR / 'cnefe2022_wide_v3.gpkg'
OUT_GEOJSON = OUT_DIR / 'cnefe2022_wide_v3.geojson'
OUT_FACE    = OUT_DIR / 'cnefe2022_aggr_face_v3.csv'
OUT_SETOR   = OUT_DIR / 'cnefe2022_aggr_setor_v3.csv'

# Leitura
gdf = gpd.read_file(INPUT_JSON)
print('CRS:', gdf.crs, '| Registros:', len(gdf))
print('Campos:', list(gdf.columns)[:20], '...')


In [ ]:

# Normalização
def as_str(x):
    return None if pd.isna(x) else str(x).strip()

for col in ['COD_UNICO_ENDERECO','COD_UF','COD_MUNICIPIO','COD_SETOR','CEP',
            'DSC_LOCALIDADE','LOGRAD_NUM','COMPLEMENTO','COD_ESPECIE','DSC_ESTABELECIMENTO']:
    if col in gdf.columns:
        gdf[col] = gdf[col].apply(as_str)

def zpad(x, n):
    if x is None:
        return None
    return x.zfill(n) if x.isdigit() else x

if 'COD_UF' in gdf.columns:        gdf['COD_UF'] = gdf['COD_UF'].apply(lambda v: zpad(v, 2) if v else v)
if 'COD_MUNICIPIO' in gdf.columns: gdf['COD_MUNICIPIO'] = gdf['COD_MUNICIPIO'].apply(lambda v: zpad(v, 7) if v else v)
if 'CEP' in gdf.columns:           gdf['CEP'] = gdf['CEP'].apply(lambda v: zpad(v, 8) if v else v)

gdf['NUM_FACE']  = gdf.get('NUM_FACE')
gdf['COD_SETOR'] = gdf.get('COD_SETOR')


In [ ]:

# Mapeamento direto (CNEFE 2022)
code_to_cat = {
    '1': 'DOMICILIO_PARTICULAR',
    '2': 'DOMICILIO_COLETIVO',
    '3': 'AGROPECUARIO',
    '4': 'ENSINO',
    '5': 'SAUDE',
    '6': 'OUTRAS_FINALIDADES',
    '7': 'CONSTRUCAO_REFORMA',
    '8': 'RELIGIOSO',
}

esp = gdf['COD_ESPECIE'].fillna('').astype(str).str.strip()

# Cria colunas cat_*
for code, name in code_to_cat.items():
    gdf[f'cat_{name}'] = (esp == code).astype('int64')

# (opcional) flag de não classificado (casos fora de 1..8)
valid = set(code_to_cat.keys())
gdf['cat_NAO_CLASSIFICADO'] = (~esp.isin(valid)).astype('int64')

gdf.filter(like='cat_').sum().sort_index()


In [ ]:

# Subtipos apenas para OUTRAS_FINALIDADES (6)
import re

def norm_text(x):
    if not x or pd.isna(x):
        return ''
    x = str(x).upper()
    tr = str.maketrans('ÁÂÃÀÄÉÊËÍÎÏÓÔÕÖÚÜÇ', 'AAAAAEEEIIIOOOOUUC')
    return x.translate(tr)

txt = gdf['DSC_ESTABELECIMENTO'].fillna('').map(norm_text)
mask_outros = gdf['cat_OUTRAS_FINALIDADES'] == 1

SUBTIPO_KEYWORDS = {
    'MERCADO': ['MERCAD', 'SUPERMERC'],
    'PADARIA': ['PADARIA', 'PANIFIC'],
    'BANCO': ['BANCO', 'BRADESCO', 'ITAU', 'SANTANDER', 'BANRISUL', 'SICREDI', 'SICOOB', 'CAIXA', 'CEF'],
    'BAR': ['BAR', 'BOTECO', 'PUB'],
    'RESTAURANTE': ['RESTAURANT', 'LANCH', 'PIZZARIA', 'CHURRASC', 'CAFETERIA'],
    'COMERCIO': ['LOJA', 'COMERC', 'ARMAZEM', 'BAZAR', 'VAREJO', 'ATACAD'],
    'SERVICOS': ['OFICINA', 'LAVACAO', 'SALAO', 'CARTOR', 'IMOBILIAR', 'CONTABIL', 'ESCRITORIO'],
    'HOSPEDAGEM': ['HOTEL', 'POUSADA', 'MOTEL', 'HOSTEL', 'PENSAO', 'PENSOES'],
    'CULTURA_LAZER': ['CLUBE', 'CINEMA', 'TEATRO', 'MUSEU', 'BIBLIOT', 'PARQUE'],
    'INDUSTRIA': ['FABRICA', 'INDUSTR', 'USINA'],
    'TRANSPORTE': ['RODOVIARIA', 'GARAGEM', 'ESTAC', 'TERMINAL'],
    'SEGURANCA': ['DELEGACIA', 'POLICIA', 'BOMBEIRO', 'BRIGADA'],
    'ADMIN_PUBLICA': ['PREFEIT', 'FORUM', 'CAMARA', 'CRAS', 'CREAS', 'INSS', 'CORREIO'],
}

# Inicializa sub_* com zeros
for key in SUBTIPO_KEYWORDS.keys():
    gdf[f'sub_{key}'] = 0

# Aplica apenas onde OUTRAS_FINALIDADES == 1
txt_outros = txt.where(mask_outros, '')
for key, pats in SUBTIPO_KEYWORDS.items():
    pat = re.compile('|'.join(map(re.escape, pats)))
    gdf.loc[txt_outros.str.contains(pat), f'sub_{key}'] = 1

# Fallback: OUTRAS sem subtipo reconhecido
gdf['sub_OUTROS'] = (mask_outros & (gdf.filter(like='sub_').sum(axis=1) == 0)).astype('int64')

gdf.filter(like='sub_').sum().sort_index()


In [ ]:

# Exportações
if gdf.crs is None:
    gdf = gdf.set_crs(4674)  # SIRGAS 2000

# GeoPackage (mantém CRS original)
gdf.to_file(OUT_GPKG, layer='cnefe2022_wide_v3', driver='GPKG')

# GeoJSON (em WGS84) — reprojecta se necessário
gdf_wgs84 = gdf.to_crs(4326) if (gdf.crs and gdf.crs.to_epsg() != 4326) else gdf
gdf_wgs84.to_file(OUT_GEOJSON, driver='GeoJSON')

print('GPKG:', OUT_GPKG)
print('GeoJSON:', OUT_GEOJSON)


In [ ]:

# Agregações estilo 2010 — somatório das colunas cat_ e sub_
dum_cols = [c for c in gdf.columns if c.startswith('cat_') or c.startswith('sub_')]

face_aggr  = gdf.groupby('NUM_FACE',  dropna=False)[dum_cols].sum().reset_index()
setor_aggr = gdf.groupby('COD_SETOR', dropna=False)[dum_cols].sum().reset_index()

face_aggr.to_csv(OUT_FACE,  index=False, encoding='utf-8')
setor_aggr.to_csv(OUT_SETOR, index=False, encoding='utf-8')

print('Agregado FACE:', OUT_FACE)
print('Agregado SETOR:', OUT_SETOR)


In [ ]:

# Resumo rápido para conferência
summary = (gdf.filter(like='cat_').sum().to_frame('total_1s')
           .merge(gdf.filter(like='sub_').sum().to_frame('total_1s'), 
                  left_index=True, right_index=True, how='outer')
           .fillna(0).astype(int))
summary.sort_index()
